In [3]:
from py_vlasov.util import zp, pade, zp_mp
from py_vlasov.util import (pmass, emass, echarge, permittivity, permeability, cspeed, boltzmann)
import numpy as np

In [4]:
def f_zeta(w, kz, vz, Omega, vthz, n):
    """
    Calculate the argument of plasma dispersion function.
    
    Keyword arguments
    -----------------
    w: frequency (rad/s)
    kz: parallel wavenumber (rad/m)
    vz: parallel drift of the particle species (m/s)
    Omega: gyrofrequency of the species (rad/s)
    vthz: parallel thermal speed (m/s)
    n: resonance number
    
    Return
    ------
    \zeta_{ns}
    """
    return (w-kz*vz-n*Omega)/(kz*vthz)

In [5]:
def choose_zp_fn(method):
    """
    choose which function to calclate the plasma dispersion function.
    
    Keyword arguments
    -----------------
    method: a string in ['pade', 'numpy', 'mpmath']
    
    Return
    ------
    return the pointer to the function object.
      
    """
    if method == 'pade':
        f_zp = pade
    elif method == 'numpy':
        f_zp = zp
    elif method == 'mpmath':
        f_zp = zp_mp
    else:
        raise VlasovException("Unreconized method.\n" +
            "Please choose between 'pade', 'numpy' and 'mpmath'")
    return f_zp

In [6]:
def r_wave_rhs(n, w, kz, kp, wp, tz, tp, vthz, vthp, Omega, vz, method = 'pade'):
    """

    Keyword arguments
    -----------------
    n: number of terms to sum over. do not need for parallel propagation.
    w: frequency
    kz: parallel wavenumber
    kp: perpendicular wavenumber. kp = 0 for parallel propagation.
    wp: plasma frequency of the species
    tz: parallel temperature
    tp: perpendicular temperature
    vthz: parallel thermal speed
    vthp: perpendicular thermal speed
    Omega: gyrofrequency
    vz: parallel drift
    
    Return
    ------
    The value of the summed term on the RHS of Eq (2), P. 267, Stix (1992). 
    Eq (2) yields R wave.
    """
    term_1 = -(tp-tz)/tz
    term_2 = ((w - kz*vz + Omega)*tp - Omega*tz)/(kz * vthz * tz)
    f_zp = choose_zp_fn(method)
    zeta = f_zeta(w, kz, vz, Omega, vthz, -1)
    term_3 = f_zp(zeta)
    rhs = (wp/w)**2 * (term_1 + term_2 * term_3)
    return rhs

In [7]:
def l_wave_rhs(n, w, kz, kp, wp, tz, tp, vthz, vthp, Omega, vz, method = 'pade'):
    """

    Keyword arguments
    -----------------
    n: number of terms to sum over. do not need for parallel propagation.
    w: frequency
    kz: parallel wavenumber
    kp: perpendicular wavenumber. kp = 0 for parallel propagation.
    wp: plasma frequency of the species
    tz: parallel temperature
    tp: perpendicular temperature
    vthz: parallel thermal speed
    vthp: perpendicular thermal speed
    Omega: gyrofrequency
    vz: parallel drift
    
    Return
    ------
    The value of an the summed term in Eq (3), P. 267, Stix (1992). 
    Eq (3) yields L wave.
    """
    term_1 = (tp-tz)/tz
    term_2 = ((w - kz*vz - Omega)*tp - Omega*tz)/(kz * vthz * tz)
    f_zp = choose_zp_fn(method)
    zeta = f_zeta(w, kz, vz, Omega, vthz, 1)
    term_3 = f_zp(zeta)
    rhs = (wp/w)**2 * (term_1 + term_2 * term_3)
    return rhs

In [8]:
def static_rhs(n, w, kz, kp, wp, tz, tp, vthz, vthp, Omega, vz, method = 'pade'):
    """

    Keyword arguments
    -----------------
    n: number of terms to sum over. do not need for parallel propagation.
    w: frequency
    kz: parallel wavenumber
    kp: perpendicular wavenumber. kp = 0 for parallel propagation.
    wp: plasma frequency of the species
    tz: parallel temperature
    tp: perpendicular temperature
    vthz: parallel thermal speed
    vthp: perpendicular thermal speed
    Omega: gyrofrequency
    vz: parallel drift
    
    Return
    ------
    The value of an the summed term in Eq (4), P. 267, Stix (1992). 
    Eq (4) yields electrostatic wave.
    """
    term_1 = 2 * (wp/ kz / vthz)**2
    term_2 = (w - kz * vz)/ (kz * vthz)
    f_zp = choose_zp_fn(method)
    zeta = f_zeta(w, kz, vz, Omega, vthz, 0)
    term_3 = f_zp(zeta)
    rhs = term_1 * (1 + term_2 * term_3)
    

In [9]:
def r_wave_eqn(param):
    """
    Keyword arguments
    -----------------
    param: a 2D list, where param[:, j] = [n_j, w, kz, kp, wp_j, tz_j, tp_j, vthz_j, vthp_j, Omega_j, vz_j, method = 'pade']
    
    Return
    ------
    Return the value of dispersion equation for R wave.    
    """
    w = param[1][0]
    kz = param[2][0]
    nz = kz * cspeed/ w
    return 1 + np.sum(np.array(list(map(r_wave_rhs, *param))), axis = 0) - nz**2

In [10]:
def l_wave_eqn(param):
    """
    Keyword arguments
    -----------------
    param: a 2D list, where param[:, j] = [n_j, w, kz, kp, wp_j, tz_j, tp_j, vthz_j, vthp_j, Omega_j, vz_j, method = 'pade']
    
    Return
    ------
    Return the value of dispersion equation for L wave.    
    """
    w = param[1][0]
    kz = param[2][0]
    nz = kz * cspeed/ w
    return 1 + np.sum(np.array(list(map(l_wave_rhs, *param))), axis = 0) - nz**2

In [11]:
def static_wave_eqn(param):
    """
    Keyword arguments
    -----------------
    param: a 2D list, where param[:, j] = [n_j, w, kz, kp, wp_j, tz_j, tp_j, vthz_j, vthp_j, Omega_j, vz_j, method = 'pade']
    
    Return
    ------
    Return the value of dispersion equation for electrostatic waves.    
    """
    return 1 + np.sum(np.array(list(map(static_rhs, *param))), axis = 0)